### Link Analysis
Crawler created using the webcrawler provided and discussed in the following jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) 


### Creating Scrappy Project
Run the following command to set up a new Scrapy project: csVillanova (Following the tutorial at https://doc.scrapy.org/en/latest/intro/tutorial.html)

scrapy startproject medium

### Create Spider
Go to the newly created folder

cd medium

Execute command for generating first spider with <B>name</B> and the <B>domain of the site</B> to be crawled

scrapy genspider news medium.com

## loading spider file

In [4]:
# %load medium/medium/spiders/news.py
import scrapy


class NewsSpider(scrapy.Spider):
    name = 'news'
    allowed_domains = ['www.olx.com.pk']
    start_urls = ['https://www.olx.com.pk/computers-accessories/']

    def parse(self, response):
        pass


Made some changed to the computer.py file so that the spider successfully crawls to next pages
+ following the tutorial at https://medium.com/python-pandemonium/develop-your-first-web-crawler-in-python-scrapy-6b2ee4baf954

In [5]:
# %load medium/medium/spiders/news.py
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
class NewsSpider(CrawlSpider):
    name = 'news'
    allowed_domains = ['www.olx.com']
    start_urls = ['https://www.olx.com/computers-accessories/','https://www.olx.com/games-entertainment/']


    rules = (
        Rule(LinkExtractor(allow=(), restrict_css=('.pageNextPrev',)),
             callback="parse_item",
             follow=True),)

    def parse_item(self, response):
        print('Processing..' + response.url)


Testing on Stackoverflow website

In [3]:
import requests
from scrapy.http import TextResponse

#running scrappy to get a response from a URL
r = requests.get('http://stackoverflow.com')
response = TextResponse(r.url, body=r.text, encoding='utf-8')

In [4]:
print response

<200 https://stackoverflow.com/>


In [5]:
#Using xpath to access data
response.xpath('//title')

[<Selector xpath='//title' data=u'<title>Stack Overflow - Where Developers'>]

In [6]:
 response.xpath('//title/text()').extract()

[u'Stack Overflow - Where Developers Learn, Share, & Build Careers']

In [10]:
response.xpath('//ul/li')

[<Selector xpath='//ul/li' data=u'<li>\r\n                            <div c'>,
 <Selector xpath='//ul/li' data=u'<li class="related-site">\r\n             '>,
 <Selector xpath='//ul/li' data=u'<li>\n                        <a href="/t'>,
 <Selector xpath='//ul/li' data=u'<li>\n                    <a href="/help"'>,
 <Selector xpath='//ul/li' data=u'<li>\n                            <a href'>,
 <Selector xpath='//ul/li' data=u'<li>\n                            <a href'>,
 <Selector xpath='//ul/li' data=u'<li>\n                            <a href'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n                <div class="favico'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n                <div class="favico'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n                <div class="favico'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n                <div class="favico'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n                <div class="favico'>,
 <Selector xpath='//ul/li' data=u'<li>\r\n   

In [11]:
response.xpath('//ul/li/a/@href').extract()

[u'https://stackoverflow.com',
 u'https://meta.stackoverflow.com',
 u'/tour',
 u'/help',
 u'https://meta.stackoverflow.com',
 u'https://stackoverflow.com/company/about',
 u'https://www.stackoverflowbusiness.com/?ref=topbar_help',
 u'https://travel.stackexchange.com/questions/105799/can-i-travel-back-to-my-homeland-after-being-granted-germany-refugee-status',
 u'https://opensource.stackexchange.com/questions/6271/calling-gpld-programs-from-a-shell-script',
 u'https://aviation.stackexchange.com/questions/46092/why-dont-aircraft-use-nuclear-propulsion',
 u'https://academia.stackexchange.com/questions/99417/how-to-prevent-students-from-using-modified-calculators-to-cheat-on-exams',
 u'https://codegolf.stackexchange.com/questions/149137/minimal-sparse-rulers',
 u'https://worldbuilding.stackexchange.com/questions/98363/if-the-romans-found-one-working-steam-engine-would-they-have-been-able-to-copy-a',
 u'https://devops.stackexchange.com/questions/2693/set-minikube-to-auto-start-on-windows-log

fetch individual item links from listing pages. I am going to modify code in parse_item method
+ fetching links by using .css method of response. (other method than xpath)
Crawling multiple pages 
following tutorial 
    + https://www.digitalocean.com/community/tutorials/how-to-crawl-a-web-page-with-scrapy-and-python-3
    + https://medium.com/python-pandemonium/develop-your-first-web-crawler-in-python-scrapy-6b2ee4baf954
top and bottom of each page has a little right carat (>) that links to the next page of results.
tell the scraper to follow that link if it exists.

In [12]:
# %load medium/medium/spiders/news.py
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from medium.items import MediumItem

class NewsSpider(CrawlSpider):
    name = "news"
    allowed_domains = ["www.olx.com.pk"]
    start_urls = [
        'https://www.olx.com.pk/computers-accessories/','https://www.olx.com.pk/games-entertainment/']

    rules = (
        Rule(LinkExtractor(allow=(), restrict_css=('.pageNextPrev',)),
             callback="parse_item",
             follow=True),)
    
    def parse_item(self, response):
        item_links = response.css('.large > .detailsLink::attr(href)').extract()
        for a in item_links:
            yield scrapy.Request(a, callback=self.parse_detail_page)



    def parse_detail_page(self, response):
        title = response.css('h1::text').extract()[0].strip()
        item = MediumItem()
        item['title'] = title
        item['url'] = response.url
        yield item

edit item.py by passing what you want to collect

In [13]:
# %load medium/medium/items.py

# Define here the models for your scraped items
#
# See documentation in:
# http://doc.scrapy.org/en/latest/topics/items.html

import scrapy
from scrapy.item import Item, Field


class MediumItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
    url = scrapy.Field()


output csv file with this command
+ scrapy crawl news -o test4.csv -t csv

In [ ]:
%load -r 1-20 medium/test4.csv 

https://www.olx.com.pk/item/zong-4g-IDWCPTz.html,Zong 4G
https://www.olx.com.pk/item/google-advertising-ad-words-video-marketing-facebook-IDWCPTJ.html,Google Advertising - Ad Words / Video Marketing - Facebook
https://www.olx.com.pk/item/hp-pavilion-15-core-i5-4th-gensilver-excellent-for-gaming-IDWCPSn.html,HP Pavilion 15 Core i5 4th Gen(Silver) Excellent For Gaming
https://www.olx.com.pk/item/dell-inspiron-15-5570-core-i5-8th-gen-4gb-1tb-amd-new-2year-warranty-IDWCPS7.html,Dell Inspiron 15 5570 Core i5 8th Gen 4GB 1TB AMD NEW 2YEAR WARRANTY
https://www.olx.com.pk/item/sapphire-hd-7850-dual-x-oc-edition-IDWCPTD.html,Sapphire hd 7850 dual-x oc edition
https://www.olx.com.pk/item/brocade-foundry-netiron-ni-ces-2024c-l3prem-ac-IDUAgLU.html,Brocade Foundry NetIron NI-CES-2024C-L3PREM-AC
https://www.olx.com.pk/item/facebook-advertising-digital-marketing-online-advertising-IDWCPU8.html,Facebook Advertising - Digital Marketing - Online Advertising


## Scrapy shell

In a command window/ terminal window

go to the tutorial project directory and type 
scrapy shell "http://www.dmoz.org/Computers/Programming/Languages/Python/Books/"

Try running the crawler to see the response using
scrapy crawl dmoz 

Let's try an example crawler that creates a csv from:

https://github.com/mjhea0/Scrapy-Samples/tree/master/crawlspider
    
I cloned the code as part of the code directory code/Scrapy-Samples-master
and added a parent URL in the output csv file it can be run by using:

cd to the code/Scrapy-Samples-master/crawlspider 
and run:
scrapy crawl craigs -o items.csv -t csv

Sample 20 lines from the csv file..

In [ ]:
# %load -r 1-20  medium/test4.csv
url,title
https://www.olx.com.pk/item/r9-280x-3gb-vapour-x-gaming-card-with-box-10-10-condition-IDWCPRv.html,R9 280x 3gb Vapour X Gaming Card With Box 10/10 Condition
https://www.olx.com.pk/item/bluetooth-keyboard-blue-x5-IDWCPYw.html,Bluetooth Keyboard Blue X5
https://www.olx.com.pk/item/dell-laptop-core-i5-3rd-generation-IDWCPNS.html,Dell Laptop core i5 3rd Generation
https://www.olx.com.pk/item/lexmark-26-29-37-color-37xl-color-ink-cartridge-IDTnPpN.html,Lexmark 26/29/37 color 37xl color ink cartridge
https://www.olx.com.pk/item/digital-marketing-social-media-manager-facebook-advertising-email-IDWCPUl.html,"Digital Marketing - Social Media manager, Facebook Advertising, email"
https://www.olx.com.pk/item/hp-led-monitor-22-inches-IDWzTBP.html,Hp led monitor 22 inches
https://www.olx.com.pk/item/bosch-pa-systems-llb4401-00-network-controller-pa-and-emergency-contro-IDUQSvR.html,Bosch PA Systems LLB4401/00 Network Controller PA and emergency contro
https://www.olx.com.pk/item/stylish-computer-trolly-IDWzTBu.html,Stylish Computer trolly
https://www.olx.com.pk/item/cisco-ws-c2950g-48-ei-ws-c2950g-24-ei-ws-c2960g-12-ei-IDUfebj.html,Cisco WS-C2950G-48-EI WS-C2950G-24-EI WS-C2960G-12-EI
https://www.olx.com.pk/item/hp-pavilion-dv6-3300se-final-price-no-negotiations-IDW2pn3.html,"HP Pavilion dv6 3300se - Final Price, No Negotiations"
https://www.olx.com.pk/item/num-pad-ibm-delivery-available-IDWCPXi.html,Num Pad IBM Delivery Available
https://www.olx.com.pk/item/lenovo-laptop-IDWCPV9.html,Lenovo laptop
https://www.olx.com.pk/item/jctech-multimedia-subwoofer-system-IDWCPO9.html,Jctech multimedia subwoofer system
https://www.olx.com.pk/item/hp-spectre-13w023-ci7-7gen-16gb-512ssd-13-3-x360-convertible-touch-IDWCPYi.html,"HP Spectre 13w023 Ci7 7gen 16gb 512ssd 13.3"" x360 Convertible Touch"
https://www.olx.com.pk/item/asus-amd-a10-5750m-5th-series-IDWCPOa.html,Asus amd A10 5750m 5th series
https://www.olx.com.pk/item/dell-vostro-14-i7-7th-gen-IDWCPOh.html,Dell Vostro 14 i7 7th Gen
https://www.olx.com.pk/item/ps4-batman-arkham-knight-region-2-exchange-possible-IDWuGvo.html,[PS4] Batman Arkham Knight region 2 exchange possible
https://www.olx.com.pk/item/overwatch-region-2-ps4-game-IDWAvx6.html,Overwatch Region 2 ps4 Game
https://www.olx.com.pk/item/sony-psp-play-station-IDWCnFk.html,Sony PSP play station

In [7]:
import pandas as pd

df = pd.read_csv('test3.csv')

In [8]:
df.head(5)

,url,title
0,https://www.olx.com.pk/item/ps4-pro-1tb-region...,Ps4 pro 1tb Region 2
1,https://www.olx.com.pk/item/evil-within-2-IDWC...,Evil within 2
2,https://www.olx.com.pk/item/overwatch-region-2...,Overwatch Region 2 ps4 Game
3,https://www.olx.com.pk/item/xbox-360-slim-500g...,Xbox 360 slim 500gb with 50 games
4,https://www.olx.com.pk/item/tritton-kama-heads...,Tritton Kama headset for ps4.


install graphistry using 
+ pip install graphistry 
+ I got the key from graphisty to visualize the graph

In [1]:
import pandas
import graphistry
graphistry.register(key='bc998b24924334858e23c47d7c05e6f286d7e4e267f95461053655516a53736e')
rows = pandas.read_csv('test4.csv')[1:100]

graphistry.hypergraph(rows)['graph'].plot()

('# links', 154)
('# event entities', 77)
('# attrib entities', 154)


### Page Rank

Using Dileep Kumars implementation: available at http://buildsearchengine.blogspot.com/2012/04/page-rank-algorithm.html

+ The compute_ranks function computes the ranks of the pages.
+ A graph, which contains pages as keys and all the links that occur in each page as their corresponding url lists. 
+ A Look_up_new function is defined that first shows the ranking of the returned url's that contain the keyword
+ urls are sorted using the QuickSort algorithm and arranged in descending order 


In [9]:
#!/usr/bin/python
#Author : dileep98490@gmail.com

import urllib
max_limit=200

def get_page(url):#This function is just to return the webpage contents; the source of the webpage when a url is given.

    try:
        f = urllib.urlopen(url)
        page = f.read()
        f.close()
        #print page
        return page
    except:	
        return ""
    return ""

In [10]:
#The union function merges the second list into first, with out duplicating an element of a, if it's already in a.
#Similar to set union operator. This function does not change b. 
#If a=[1,2,3] b=[2,3,4]. After union(a,b) makes a=[1,2,3,4] and b=[2,3,4]
def union(a,b):
    for e in b:
        if e not in a:
            a.append(e)

def get_next_url(page):
    start_link=page.find("a href")
    if(start_link==-1):
        return None,0

    start_quote=page.find('"',start_link)
    end_quote=page.find('"',start_quote+1)
    url=page[start_quote+1:end_quote]
    return url,end_quote

def get_all_links(page):
    links=[]
    while(True):
        url,n=get_next_url(page)
        page=page[n:]
        if url:
            links.append(url)
        else:
            break
    return links

In [11]:
#This function is for given an index, it finds the keyword in the index and returns the list of links
def Look_up(index,keyword):
    #f=[]
    if keyword in index:
        return index[keyword]
    return []

#The format of element in the index is <keyword>,[<List of urls that contain the keyword>]
def add_to_index(index,url,keyword):
    if keyword in index:
        if url not in index[keyword]:
            index[keyword].append(url)
        return
    index[keyword]=[url]

#Adding the content of the webpage to the index
def add_page_to_index(index,url,content):
    for i in content.split():
        add_to_index(index,url,i)

In [12]:
#Computing ranks for a given graph -> for all the links in it
def compute_ranks(graph):
    d=0.8
    numloops=10
    ranks={}
    npages=len(graph)

    for page in graph:
        ranks[page]=1.0/npages

    for i in range(0,numloops):
        newranks={}
        for page in graph:
            newrank=(1-d)/npages
            for node in graph:
                if page in graph[node]:
                    newrank=newrank+d*ranks[node]/len(graph[node])
            newranks[page]=newrank
        ranks=newranks
    return ranks

def Crawl_web(seed):#The website to act as seed page is given as input

    tocrawl=[seed]
    crawled=[]
    index={}
    graph={}#new graph
    global max_limit

    while tocrawl:
        p=tocrawl.pop()
        #To remove the looping, if a page is already crawled and it is backlinked again by someother link we are crawling, we need not crawl it again
        if p not in crawled:
            max_limit-=1
            print max_limit
            if max_limit<=0:
                break
            c=get_page(p)
            add_page_to_index(index,p,c)
            f=get_all_links(c)
            union(tocrawl,f)
            graph[p]=f
            crawled.append(p)#As soon as a link is crawled it is appended to crawled. In the end when all the links are over, we will return the crawled since it contains all the links we have so far

    return crawled,index,graph #Returns the list of links

In [13]:
def QuickSort(pages,ranks):#Sorting in descending order

    if len(pages)>1:
        piv=ranks[pages[0]]
        i=1
        j=1
        for j in range(1,len(pages)):
            if ranks[pages[j]]>piv:
                pages[i],pages[j]=pages[j],pages[i]
                i+=1
        pages[i-1],pages[0]=pages[0],pages[i-1]
        QuickSort(pages[1:i],ranks)
        QuickSort(pages[i+1:len(pages)],ranks)



def Look_up_new(index,ranks,keyword):

    pages=Look_up(index,keyword)
    print '\nPrinting the results as is with page rank\n'
    for i in pages:
        print i+" --> "+str(ranks[i])#Displaying the lists, so that you can see the page rank along side

    QuickSort(pages,ranks)
    print "\nAfter Sorting the results by page rank\n"
    it=0
    for i in pages:#This is how actually it looks like in search engine results - > sorted by page rank
        it+=1
        print str(it)+'.\t'+i+'\n' 

In [14]:
#print index
print "Enter the seed page"
seed_page=raw_input()

print "Enter What you want to search"
search_term=raw_input()

try:
    print "Enter the depth you wanna go"
    max_limit=int(raw_input())
except:
    f=None
print '\nStarted crawling, presently at depth..'

crawled,index,graph=Crawl_web(seed_page)#printing all the links
ranks=compute_ranks(graph)#Calculating the page ranks
Look_up_new(index,ranks,search_term)

Enter the seed page
https://www.olx.com.pk/computers-accessories/
Enter What you want to search
is
Enter the depth you wanna go
12

Started crawling, presently at depth..
11
10
9
8
7
6
5
4
3
2
1
0

Printing the results as is with page rank

https://www.olx.com.pk/computers-accessories/ --> 0.0181818181818
https://www.olx.com.pk/myaccount/ --> 0.0214974217391
https://www.olx.com.pk/account/register/ --> 0.0196792399209
https://itunes.apple.com/app/olx-pakistan/id1119081665?mt=8 --> 0.0196792399209
http://eclipse.org/jetty --> 0.0329605081645
https://www.eclipse.org --> 0.0189000855001
https://www.linkedin.com/company/eclipse-foundation --> 0.0185402406478
https://status.eclipse.org --> 0.0183732114527
https://wiki.eclipse.org/IT_Infrastructure_Doc#Use_mirror_sites.2Fsee_which_mirrors_are_mirroring_my_files.3F --> 0.0183488473768

After Sorting the results by page rank

1.	https://wiki.eclipse.org/IT_Infrastructure_Doc#Use_mirror_sites.2Fsee_which_mirrors_are_mirroring_my_files.3F

2.	ht